<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/Spark_lab_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [2]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2026-01-10 01:56:06--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  24.9MB/s    in 21s     

2026-01-10 01:56:27 (25.3 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [3]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


# 🟢 LAB 2 – PARTITION & FILE (NỀN TẢNG SỐNG CÒN)

## Mục tiêu LAB 2

*	Hiểu partition là gì (80% performance Spark nằm ở đây)
*	Hiểu Spark đọc file khác Pandas thế nào
*	Biết khi nào nên tăng / giảm partition
*	Không tuning mù

---

## 0️⃣ Nhắc lại tư duy (rất quan trọng)

* Spark không xử lý theo dòng
* Spark xử lý theo PARTITION

> 📦 **Partition** = đơn vị song song nhỏ nhất của **Spark**

---

## 1️⃣ Kiểm tra số partition hiện tại

**Dùng lại DataFrame từ LAB 1:**

```python
df.rdd.getNumPartitions()
```

> 👉 Kết quả thường là 1 (trên Colab / local)

### 🧠 Ý nghĩa
*	1 partition = 1 task
*	1 core chạy
*	Spark lúc này không nhanh hơn Pandas

---

## 2️⃣ Tăng partition – bước đầu song song

```python
df2 = df.repartition(4)
df2.rdd.getNumPartitions()
```

### 📌 Output: 4

#### 🧠 Điều gì xảy ra?
*	Spark sẽ chia data thành 4 phần
*	4 task
*	Có thể chạy song song nếu có core

#### ⛔ LƯU Ý
*	repartition() = SHUFFLE
*	Dùng bừa = tốn tiền

---

## 3️⃣ Coalesce – giảm partition KHÔNG shuffle

```python
df3 = df2.coalesce(2)
df3.rdd.getNumPartitions()
```

### 📌 Output: 2

#### 🧠 Khác gì repartition?

|**repartition**|**coalesce**|
|---------------|------------|
|Có shuffle|Không shuffle|
|Tốn tiền|Rẻ|
|Dùng khi tăng|Dùng khi giảm|


---

## 4️⃣ Khi nào partition là “đẹp”?

#### Quy tắc thực tế (industry rule)

> Số **partition** ≈ **(Data size / 128MB)**

#### Ví dụ:
*	10GB → ~80 partitions
*	1TB → ~8000 partitions

> 📌 KHÔNG dùng mặc định 200 cho mọi job

---

## 5️⃣ Đọc file – Spark khác Pandas thế nào?

### Pandas

```python
pd.read_csv("big_file.csv")
```

> ⛔ Đọc toàn bộ file vào RAM → chết

---

### Spark

```python
spark.read.csv("big_file.csv")
```

#### 🧠 Spark:
*	Không đọc ngay
*	Chia file thành block
*	Mỗi block → partition
*	Chỉ đọc khi có ACTION

---

## 6️⃣ Tạo file test (nhẹ, không cần GCS)

```python
df_big = spark.range(0, 1_000_000) \
    .withColumn("value", F.rand())

df_big.rdd.getNumPartitions()
```

> 👉 thường = 1 (local)

---

## 7️⃣ Ghi file – PARTITION ẢNH HƯỞNG FILE OUTPUT

```python
df_big.write.mode("overwrite").parquet("/tmp/lab2_parquet")
```

### 📌 Kết quả:
* 1 partition → 1 file parquet

---

### Tăng partition rồi ghi

```python
df_big.repartition(4) \
    .write.mode("overwrite") \
    .parquet("/tmp/lab2_parquet_4")
```

#### 📌 Kết quả:
*	4 file parquet

#### 🧠 Bài học cực quan trọng

* Số partition = số file output

* ⛔ Too many files = NameNode / metadata chết

---

## 8️⃣ Kiểm tra lại khi đọc file

```python
df_read = spark.read.parquet("/tmp/lab2_parquet_4")
df_read.rdd.getNumPartitions()
```

👉 Spark giữ nguyên partition theo file

---

## 9️⃣ Tổng kết LAB 2 (cực kỳ quan trọng)

### ✅ Bạn phải nhớ:
*	Partition = đơn vị song song
*	Repartition = shuffle
*	Coalesce = giảm partition không shuffle
*	Số partition ảnh hưởng:
-	Speed
-	Cost
-	File output
-	Stability

---

## 🧠 CÂU HỎI BẮT BUỘC (trả lời trước LAB 3)
1.	Vì sao tăng partition không phải lúc nào cũng nhanh hơn?
2.	Vì sao repartition tốn tiền?
3.	Khi nào nên dùng coalesce?
4.	Vì sao quá nhiều file output là thảm họa?
5.	Spark đọc file lúc nào?



In [4]:
data = [
    (1, "Alice", 1000),
    (2, "Bob", 2000),
    (3, "Charlie", 3000),
    (4, "David", 4000),
    (5, "Eve", 5000),
    (6, "Frank", 6000),
    (7, "Grace", 7000)
]

columns = ["id", "name", "balance"]

df = spark.createDataFrame(data, columns)

In [5]:
df.rdd.getNumPartitions()

2

In [6]:
df2 = df.repartition(4)
df2.rdd.getNumPartitions()

4

In [7]:
df3 = df2.coalesce(2)
df3.rdd.getNumPartitions()

2

In [8]:
df_big = spark.range(0, 1_000_000) \
    .withColumn("value", F.rand())

In [9]:
df_big.rdd.getNumPartitions()

2

In [10]:
df_big.write.mode("overwrite").parquet("/tmp/lab2_parquet")

In [11]:
df_big.repartition(4) \
    .write.mode("overwrite") \
    .parquet("/tmp/lab2_parquet_4")

In [12]:
df_read = spark.read.parquet("/tmp/lab2_parquet_4")
df_read.rdd.getNumPartitions()

2